## Initialization
This file fixed the FIPS of the Precipitation and Temperature Datasets, since it was not correct. FIPS is a 5 digit unique identifier for every county.
digits 1-2 represent the state
digits 3-5 represent the county
Here, we fix the digits 1 and 2 to correctly represent the state.
  Range of values of 01-50 for the states in the datasets

                             01 Alabama                 28 New Jersey
                             02 Arizona                 29 New Mexico
                             03 Arkansas                30 New York
                             04 California              31 North Carolina
                             05 Colorado                32 North Dakota
                             06 Connecticut             33 Ohio
                             07 Delaware                34 Oklahoma
                             08 Florida                 35 Oregon
                             09 Georgia                 36 Pennsylvania
                             10 Idaho                   37 Rhode Island
                             11 Illinois                38 South Carolina
                             12 Indiana                 39 South Dakota
                             13 Iowa                    40 Tennessee
                             14 Kansas                  41 Texas
                             15 Kentucky                42 Utah
                             16 Louisiana               43 Vermont
                             17 Maine                   44 Virginia
                             18 Maryland                45 Washington
                             19 Massachusetts           46 West Virginia
                             20 Michigan                47 Wisconsin
                             21 Minnesota               48 Wyoming
                             22 Mississippi
                             23 Missouri   
                             24 Montana   
                             25 Nebraska 
                             26 Nevada  
                             27 New Hampshire

In the two Datasets, the FIPS were not represented according to the official FIPS representation in the USA, this would cause us issues when using K means and other ML algorithms, therefore the correct FIPS should be represented in the data. This file will tackle this problem and will export the new datasets which will be used in M2.ipynb

In [ ]:
############################## only for google colab #################################

from google.colab import drive
drive.mount("/content/drive")

######################################################################################

Mounted at /content/drive


In [ ]:
import numpy as numpy
import pandas as pandas
import matplotlib.pyplot as plt
# import seaborn as sns

from IPython.display import HTML, display
import tabulate



## Fixing the Precipitation Dataset

In [ ]:
df_precipitation = pandas.read_excel("/content/drive/My Drive/CMPS_276/data/Precipitation_Data_Per_Month_1895to2021_Excel.xlsx")
print(df_precipitation.head())

    encodings  State and County  ...  November  December
0  1001011895              1001  ...      1.44      3.66
1  1001011896              1001  ...      2.89      1.94
2  1001011897              1001  ...      1.84      4.38
3  1001011898              1001  ...      6.66      3.91
4  1001011899              1001  ...      1.98      5.25

[5 rows x 16 columns]


In [ ]:
print(df_precipitation.tail())

          encodings  State and County  ...  November  December
397397  50290012017             50290  ...      1.42      0.82
397398  50290012018             50290  ...      0.95      0.88
397399  50290012019             50290  ...      2.25      0.58
397400  50290012020             50290  ...      1.12      0.52
397401  50290012021             50290  ...     -9.99     -9.99

[5 rows x 16 columns]


In [ ]:
#editing the fips
#this dataset contains the wrong fips encoded in the precipitation and the temperature datasets
df_FIPS = pandas.read_excel("/content/drive/My Drive/CMPS_276/data/WrongFIPS.xlsx")
df_FIPS.head()
#creating new column target FIPS that will be the FIPS I need
#df_FIPS["TargetFIPS"]=df_FIPS['code']
#df_FIPS.head()

,code,State
0,1,ALABAMA
1,2,ARIZONA
2,3,ARKANSAS
3,4,CALIFORNIA
4,5,COLORADO


In [ ]:
df_FIPS_real = pandas.read_csv("/content/drive/My Drive/CMPS_276/data/FIPS and COUNTIES.csv")
print(df_FIPS_real.head())

#removing duplicates in State Column to get unique states and their first 2 digit FIPS
df_FIPS_real.drop_duplicates(subset ="State", keep = 'first', inplace = True)
print(df_FIPS_real.head()) 

   FIPS   COUNTY    State
0  1001  AUTAUGA  ALABAMA
1  1003  BALDWIN  ALABAMA
2  1005  BARBOUR  ALABAMA
3  1007     BIBB  ALABAMA
4  1009   BLOUNT  ALABAMA
      FIPS     COUNTY           State
0     1001    AUTAUGA         ALABAMA
67    2013  ALEUTIANS          ALASKA
96   60010    EASTERN  AMERICAN SAMOA
101   4001     APACHE         ARIZONA
116   5001   ARKANSAS        ARKANSAS


In [ ]:
st = ""
st_code = 0
list1=[]
#print(df_FIPS_real[df_FIPS_real["State"]=="ALABAMA"]["FIPS"])
#Loop over all states in the real_FIPS dataset, and for every state, get the correct code which represents it
for state in df_FIPS_real["State"]:
  st = state
  fips = df_FIPS_real[df_FIPS_real["State"]==state]["FIPS"].values.tolist()
  #if fips has 5 digits, then take the first 2 digits, since they represent the state
  # else, take the fisrt digit which represents the state. 
  if len(str(fips[-1]))>4:
    st_code = int(str(fips[-1])[:2])
  else:
    st_code = int(str(fips[-1])[:1])
  for state2 in df_FIPS["State"]:
    if state2 == state:
      temp = str(st_code)+str(df_FIPS_real[df_FIPS_real["State"]==state]["FIPS"])[-3:]
      #print(st_code)
      #print(df_FIPS[df_FIPS["State"]==state]["TargetFIPS"])
      #df_FIPS[df_FIPS["State"]==state]["TargetFIPS"] = st_code

      #appending all correct codes in a list, to be later added as a column in the dataframe df_FIPS
      list1.append(st_code)
      #print(df_FIPS[df_FIPS["State"]==state]["TargetFIPS"])
      #print(df_FIPS[df_FIPS["State"]==state]["TargetFIPS"])
      #df_FIPS["TargetFIPS"].replace({df_FIPS[df_FIPS["State"]==state]["TargetFIPS"]: st_code}, inplace=True)

#creating a data frame from the list of correct codes
df_temp1 = pandas.DataFrame(list1, columns=['target_fips'])
#adding a new column in the df_FIPS data frame
df_FIPS["TargetFIPS"] = df_temp1['target_fips']
new_row = {'code':50, 'State':"ALASKA", 'TargetFIPS':2}
#append row to the dataframe, adding ALASKA since it was missing.
df_FIPS = df_FIPS.append(new_row, ignore_index=True)
print(df_FIPS)


def GetNewCode(oldCode):
  """
  This function gets the old incorrect code representing a State and gives back the correct code that represents the state
  """
  n = len(df_FIPS['code']) 
  for i in range(n):
    if df_FIPS['code'].iloc[[i]].values == oldCode:
      return int(df_FIPS['TargetFIPS'].iloc[[i]].values)

#print(GetNewCode(9))

    code           State  TargetFIPS
0      1         ALABAMA           1
1      2         ARIZONA           4
2      3        ARKANSAS           5
3      4      CALIFORNIA           6
4      5        COLORADO           8
5      6     CONNECTICUT           9
6      7        DELAWARE          10
7      8         FLORIDA          12
8      9         GEORGIA          13
9     10           IDAHO          16
10    11        ILLINOIS          17
11    12         INDIANA          18
12    13            IOWA          19
13    14          KANSAS          20
14    15        KENTUCKY          21
15    16       LOUISIANA          22
16    17           MAINE          23
17    18        MARYLAND          24
18    19   MASSACHUSETTS          25
19    20        MICHIGAN          26
20    21       MINNESOTA          27
21    22     MISSISSIPPI          28
22    23        MISSOURI          29
23    24         MONTANA          30
24    25        NEBRASKA          31
25    26          NEVADA          32
2

In [ ]:
#Running this cell will take around 25 mins
n = len(df_precipitation['State and County'])
for i in range(n):
  #get old fips
  temp1 = int(df_precipitation['State and County'].iloc[[i]].values)
  print(int(temp1))
  #get last 3 digits of old FIPS
  temp2 = str(int(df_precipitation['State and County'].iloc[[i]].values))[-3:]
  #print(temp2)
  if len(str(temp1))>4:
    #get first 2 digits of old fips
    old_code = int(str(temp1)[:2])
    #get new digits of old fips
    new_code = GetNewCode(old_code)
    #replace old by new fips
    temp3 = int(str(new_code)+temp2)
    df_precipitation['State and County'].iloc[i] = temp3
  else:
    #get first digit of old fips
    old_code = int(str(temp1)[:1])
    #get new digits of old fips
    new_code = GetNewCode(old_code)
    #replace old by new fips
    temp3 = int(str(new_code)+temp2)
    #print(temp3)
    df_precipitation['State and County'].iloc[i] = temp3

print(df_precipitation.tail())

1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1003
1003
1003
1003
1003


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Streaming output truncated to the last 5000 lines.
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48015
48015
48015
48015
48015
48015
48015
48015
48015
48015
48015
4

In [ ]:
#merging FIPS and year to have a unique identifier for every row
#new column ID_precipitation is a unique identifier for every row
df_precipitation["ID_precipitation"] = df_precipitation["Year"].astype(str) + df_precipitation["State and County"].astype(str)
df_precipitation["ID_precipitation"] = df_precipitation["ID_precipitation"].astype(int)
df_precipitation = df_precipitation.drop(columns=['encodings'])
df_precipitation.describe()

,State and County,code(1_indicates_precipitation_values),Year,January,February,March,April,May,June,July,August,September,October,November,December,ID_precipitation
count,397402.000000,397402.0,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,3.974020e+05
mean,30467.545183,1.0,1958.106177,2.764770,2.610404,3.220008,3.260575,3.713410,3.796031,3.717997,3.460788,3.214601,2.672857,2.617541,2.788858,1.784216e+08
std,15112.538987,0.0,36.628142,2.642054,2.346546,2.481572,2.142210,2.242912,2.286095,2.382784,2.332756,2.466514,2.612978,2.697771,2.882478,5.270258e+07
min,1001.000000,1.0,1895.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.990000,-9.990000,-9.990000,1.895100e+07
25%,19013.000000,1.0,1926.000000,0.870000,0.900000,1.380000,1.720000,2.100000,2.200000,2.000000,1.840000,1.490000,1.150000,1.010000,0.960000,1.916230e+08
50%,29186.000000,1.0,1958.000000,2.110000,2.000000,2.700000,2.900000,3.390000,3.530000,3.420000,3.090000,2.710000,2.240000,2.170000,2.270000,1.951360e+08
75%,45085.000000,1.0,1990.000000,3.850000,3.650000,4.430000,4.340000,4.930000,5.050000,5.050000,4.640000,4.320000,3.740000,3.680000,3.970000,1.986471e+08
max,56045.000000,1.0,2021.000000,38.040000,34.450000,35.420000,25.920000,24.660000,23.850000,28.090000,46.040000,51.440000,59.400000,54.800000,44.760000,2.021560e+08


In [ ]:
#now let's export the pandas dataframe into a CSV file, sincce it took 25 mins to fix the FIPS.
df_precipitation.to_csv(r'/content/drive/My Drive/CMPS_276/data/FIXED_FIPS_Precipitation_Data_PerMonth_1895to2020.csv', index = False)

## Fixing the Temperature Dataset

In [ ]:
df_temperature = pandas.read_excel("/content/drive/My Drive/CMPS_276/data/Temperature_Data_1895to2021.xlsx")
print(df_temperature.head())
print(df_temperature.tail())

   FIPS  Temperature code  Year  ...  October  November  December
0  1001                 2  1895  ...     61.4      54.4      45.3
1  1002                 2  1895  ...     61.4      54.4      45.3
2  1003                 2  1895  ...     61.4      54.4      45.3
3  1004                 2  1895  ...     61.4      54.4      45.3
4  1005                 2  1895  ...     61.4      54.4      45.3

[5 rows x 15 columns]
         FIPS  Temperature code  Year  ...  October  November  December
397397  50290                 2  2017  ...     27.5       8.1      14.2
397398  50290                 2  2018  ...     30.4      11.7       2.7
397399  50290                 2  2019  ...     27.5      10.7      -4.9
397400  50290                 2  2020  ...     24.8       5.9       3.1
397401  50290                 2  2021  ...     25.8     -99.9     -99.9

[5 rows x 15 columns]


In [ ]:
df_temperature.describe()

,FIPS,Temperature code,Year,January,February,March,April,May,June,July,August,September,October,November,December
count,397402.000000,397402.0,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000
mean,25058.154290,2.0,1958.098024,31.670790,34.648519,43.308130,53.290537,62.641730,70.873090,75.073276,73.679860,66.950445,55.898139,42.674133,33.444635
std,13822.594289,0.0,36.636214,13.095281,12.959376,11.442292,9.234442,8.139247,7.412473,6.351007,6.773659,7.822304,8.492353,16.112234,16.699450
min,1001.000000,2.0,1895.000000,-29.400000,-31.700000,-22.600000,-6.800000,17.400000,37.800000,43.700000,37.400000,23.600000,0.600000,-99.900000,-99.900000
25%,13057.000000,2.0,1926.000000,22.800000,25.600000,35.200000,46.600000,57.000000,66.200000,71.200000,69.200000,61.500000,50.000000,36.900000,26.300000
50%,24001.000000,2.0,1958.000000,32.000000,35.000000,43.300000,53.300000,63.000000,71.600000,75.800000,74.100000,67.300000,55.900000,43.700000,34.500000
75%,39039.000000,2.0,1990.000000,41.200000,44.100000,51.800000,60.400000,68.900000,76.600000,79.900000,78.900000,72.900000,62.000000,50.700000,42.800000
max,50290.000000,2.0,2021.000000,73.600000,75.500000,77.500000,81.700000,86.000000,92.200000,96.200000,96.600000,90.300000,82.600000,79.200000,77.600000


In [ ]:
#running this cell took 24 mins
n = len(df_temperature['FIPS'])
for i in range(n):
  #get old fips
  temp1 = int(df_temperature['FIPS'].iloc[[i]].values)
  print(int(temp1))
  #get last 3 digits of old FIPS
  temp2 = str(int(df_temperature['FIPS'].iloc[[i]].values))[-3:]
  #print(temp2)
  if len(str(temp1))>4:
    #get first 2 digits of old fips
    old_code = int(str(temp1)[:2])
    #get new digits of old fips
    new_code = GetNewCode(old_code)
    #replace old by new fips
    temp3 = int(str(new_code)+temp2)
    df_temperature['FIPS'].iloc[i] = temp3
  else:
    #get first digit of old fips
    old_code = int(str(temp1)[:1])
    #get new digits of old fips
    new_code = GetNewCode(old_code)
    #replace old by new fips
    temp3 = int(str(new_code)+temp2)
    #print(temp3)
    df_temperature['FIPS'].iloc[i] = temp3

print(df_temperature.tail())

1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001
1001


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Streaming output truncated to the last 5000 lines.
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48011
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48013
48015
48015
48015
48015
48015
48015
48015
48015
48015
48015
48015
4

In [ ]:
#merging FIPS and year to have a unique identifier for every row
#new column ID_temperature is a unique identifier for every row
df_temperature["ID_temperature"] = df_temperature["Year"].astype(str) + df_temperature["FIPS"].astype(str)
df_temperature["ID_temperature"] = df_temperature["ID_temperature"].astype(int)
#df_temperature = df_temperature.drop(columns=['FIPS'])
df_temperature.describe()

,FIPS,Temperature code,Year,January,February,March,April,May,June,July,August,September,October,November,December,ID_temperature
count,397402.000000,397402.0,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,397402.000000,3.974020e+05
mean,30467.553336,2.0,1958.098024,31.670790,34.648519,43.308130,53.290537,62.641730,70.873090,75.073276,73.679860,66.950445,55.898139,42.674133,33.444635,1.784215e+08
std,15112.523105,0.0,36.636214,13.095281,12.959376,11.442292,9.234442,8.139247,7.412473,6.351007,6.773659,7.822304,8.492353,16.112234,16.699450,5.270283e+07
min,1001.000000,2.0,1895.000000,-29.400000,-31.700000,-22.600000,-6.800000,17.400000,37.800000,43.700000,37.400000,23.600000,0.600000,-99.900000,-99.900000,1.895100e+07
25%,19013.000000,2.0,1926.000000,22.800000,25.600000,35.200000,46.600000,57.000000,66.200000,71.200000,69.200000,61.500000,50.000000,36.900000,26.300000,1.916230e+08
50%,29186.000000,2.0,1958.000000,32.000000,35.000000,43.300000,53.300000,63.000000,71.600000,75.800000,74.100000,67.300000,55.900000,43.700000,34.500000,1.951360e+08
75%,45085.000000,2.0,1990.000000,41.200000,44.100000,51.800000,60.400000,68.900000,76.600000,79.900000,78.900000,72.900000,62.000000,50.700000,42.800000,1.986471e+08
max,56045.000000,2.0,2021.000000,73.600000,75.500000,77.500000,81.700000,86.000000,92.200000,96.200000,96.600000,90.300000,82.600000,79.200000,77.600000,2.021560e+08


In [ ]:
#now let's export the pandas dataframe into a CSV file, sincce it took 25 mins to fix the FIPS.
df_temperature.to_csv(r'/content/drive/My Drive/CMPS_276/data/FIXED_FIPS_Temperature_Data_PerMonth_1895to2020.csv', index = False)